In [17]:
from __future__ import print_function, division
import numpy as np 
import pandas as pd 

import os
        
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

import optuna
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import xgboost as xgb
torch.manual_seed(15)

# Import data

In [4]:
df_classes = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
df_edges = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
df_features = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)
df_features.columns = ['txId', 'time_step'] + [f'trans_feat_{i}' for i in range(93)] + [f'agg_feat_{i}' for i in range(72)]
df_features = pd.merge(df_features,df_classes,left_on="txId",right_on="txId",how='left')
df_features['class'] = df_features['class'].apply(lambda x: '0' if x == "unknown" else x)

# Train and test data

In [7]:
tx_features = ["trans_feat_"+str(i) for i in range(93)]
agg_features = ["agg_feat_"+str(i) for i in range(72)]
data = df_features[(df_features['class']=='1') | (df_features['class']=='2')]
X = data[tx_features + agg_features] 
y = data['class']
y = y.apply(lambda x: 0 if x == '2' else 1 )
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)

# Fine-tuning

In [12]:
rsn=15
kf = StratifiedKFold(n_splits=3)

def objective_xgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 20, 320)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    gamma = trial.suggest_float('gamma', 0, 1)
    subsample = trial.suggest_float('subsample', 0.1, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 1.0)

    params = {
        'max_depth': max_depth,
        'learning_rate': learning_rate,
        'min_child_weight': min_child_weight,
        'gamma': gamma,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree
    }
    model = xgb.XGBClassifier(n_estimators=n_estimators, **params)
    f1_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='f1_micro')
    return f1_scores.mean()

def print_trial_callback(study, trial):
    print(f"Trial {trial.number}: {trial.value}")
    print(f"Params: {trial.params}")

In [14]:
#XGB
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=30, callbacks=[print_trial_callback])

xgb_best_params = study_xgb.best_trial.params
xgbc = xgb.XGBClassifier(**xgb_best_params, random_state=rsn).fit(X_train, y_train)
xgb_preds = xgbc.predict(X_test)
xgb_prec, xgb_rec, xgb_f1, xgb_num = precision_recall_fscore_support(y_test, xgb_preds, average=None)


[I 2023-10-04 12:22:30,022] A new study created in memory with name: no-name-c87fc4b3-d2d3-417f-80cf-5e5363b7ca92
[I 2023-10-04 12:22:33,356] Trial 0 finished with value: 0.8875253111952577 and parameters: {'n_estimators': 53, 'max_depth': 10, 'learning_rate': 0.007251779774762484, 'min_child_weight': 4, 'gamma': 0.8848518863952797, 'subsample': 0.7484253696091445, 'colsample_bytree': 0.9073313258780329}. Best is trial 0 with value: 0.8875253111952577.


Trial 0: 0.8875253111952577
Params: {'n_estimators': 53, 'max_depth': 10, 'learning_rate': 0.007251779774762484, 'min_child_weight': 4, 'gamma': 0.8848518863952797, 'subsample': 0.7484253696091445, 'colsample_bytree': 0.9073313258780329}


[I 2023-10-04 12:22:38,762] Trial 1 finished with value: 0.9651160050795218 and parameters: {'n_estimators': 219, 'max_depth': 15, 'learning_rate': 0.008170108539597468, 'min_child_weight': 18, 'gamma': 0.14053849704246602, 'subsample': 0.2636346810474465, 'colsample_bytree': 0.20412117985510647}. Best is trial 1 with value: 0.9651160050795218.


Trial 1: 0.9651160050795218
Params: {'n_estimators': 219, 'max_depth': 15, 'learning_rate': 0.008170108539597468, 'min_child_weight': 18, 'gamma': 0.14053849704246602, 'subsample': 0.2636346810474465, 'colsample_bytree': 0.20412117985510647}


[I 2023-10-04 12:22:43,648] Trial 2 finished with value: 0.8197733525498481 and parameters: {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.06510708583141818, 'min_child_weight': 18, 'gamma': 0.5634243875273641, 'subsample': 0.625582662318, 'colsample_bytree': 0.9868717705666786}. Best is trial 1 with value: 0.9651160050795218.


...

Trial 27: 0.8846665524655357
Params: {'n_estimators': 247, 'max_depth': 7, 'learning_rate': 0.01679376231043707, 'min_child_weight': 12, 'gamma': 0.0669876491439647, 'subsample': 0.23784557524177521, 'colsample_bytree': 0.386027025758811}


[I 2023-10-04 12:24:42,400] Trial 28 finished with value: 0.8875253111952577 and parameters: {'n_estimators': 147, 'max_depth': 11, 'learning_rate': 0.003003097829738819, 'min_child_weight': 5, 'gamma': 0.4130089068956302, 'subsample': 0.16672840902935382, 'colsample_bytree': 0.19529781039052085}. Best is trial 4 with value: 0.9753331120474624.


Trial 28: 0.8875253111952577
Params: {'n_estimators': 147, 'max_depth': 11, 'learning_rate': 0.003003097829738819, 'min_child_weight': 5, 'gamma': 0.4130089068956302, 'subsample': 0.16672840902935382, 'colsample_bytree': 0.19529781039052085}


[I 2023-10-04 12:24:50,901] Trial 29 finished with value: 0.7714178686025058 and parameters: {'n_estimators': 320, 'max_depth': 9, 'learning_rate': 0.006409483993872512, 'min_child_weight': 3, 'gamma': 0.16575748365846027, 'subsample': 0.272721157841323, 'colsample_bytree': 0.29922229156982094}. Best is trial 4 with value: 0.9753331120474624.


Trial 29: 0.7714178686025058
Params: {'n_estimators': 320, 'max_depth': 9, 'learning_rate': 0.006409483993872512, 'min_child_weight': 3, 'gamma': 0.16575748365846027, 'subsample': 0.272721157841323, 'colsample_bytree': 0.29922229156982094}


XGB Classifier Model
Precision: 0.990 
Recall: 0.568 
F1 Score: 0.722
Micro-Average F1 Score: 0.9724409448818898


In [20]:
# Evaluation
print("\n\nXGB Classifier Model")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f" % (xgb_prec[1], xgb_rec[1], xgb_f1[1]))
micro_f1 = f1_score(y_test, xgb_preds, average='micro')
print("Micro-Average F1 Score:", micro_f1)
classification = classification_report(y_test, xgb_preds)
print("Classification Report: \n", classification)



XGB Classifier Model
Precision: 0.990 
Recall: 0.568 
F1 Score: 0.722
Micro-Average F1 Score: 0.9724409448818898
Classification Report: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.99     13091
           1       0.99      0.57      0.72       879

    accuracy                           0.97     13970
   macro avg       0.98      0.78      0.85     13970
weighted avg       0.97      0.97      0.97     13970



In [16]:
# Best params
xgb_best_params

{'n_estimators': 275,
 'max_depth': 9,
 'learning_rate': 0.005221004727527383,
 'min_child_weight': 1,
 'gamma': 0.060071690764745944,
 'subsample': 0.267771515127166,
 'colsample_bytree': 0.19908627339554574}

# Embedding

In [21]:
#Then add embedding features and run it all over again...

embed_names = ["emb_"+str(i) for i in range(1,51)]
embeddings = pd.read_csv('elliptic_bitcoin_dataset/elliptic.emb',delimiter=" ",skiprows=1,header=None)
embeddings.columns = ['txId'] + ["emb_"+str(i) for i in range(1,51)]

data = df_features[(df_features['class']=='1') | (df_features['class']=='2')]
data = pd.merge(data,embeddings,how='inner')
X = data[tx_features+agg_features+embed_names]
y = data['class']
y = y.apply(lambda x: 0 if x == '2' else 1 )
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)

In [25]:
#XGB
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=20, callbacks=[print_trial_callback])

xgb_best_params = study_xgb.best_trial.params
xgbc = xgb.XGBClassifier(**xgb_best_params, random_state=rsn).fit(X_train, y_train)
xgb_preds = xgbc.predict(X_test)
xgb_prec, xgb_rec, xgb_f1, xgb_num = precision_recall_fscore_support(y_test, xgb_preds, average=None)

[I 2023-10-04 12:36:48,143] A new study created in memory with name: no-name-aeee6744-0132-48a4-a1e7-ecc2e5a7389c


[I 2023-10-04 12:37:00,856] Trial 0 finished with value: 0.887473525678869 and parameters: {'n_estimators': 249, 'max_depth': 14, 'learning_rate': 0.00033095825586489704, 'min_child_weight': 20, 'gamma': 0.35714827378744807, 'subsample': 0.8450006684154654, 'colsample_bytree': 0.8875084020743679}. Best is trial 0 with value: 0.887473525678869.


Trial 0: 0.887473525678869
Params: {'n_estimators': 249, 'max_depth': 14, 'learning_rate': 0.00033095825586489704, 'min_child_weight': 20, 'gamma': 0.35714827378744807, 'subsample': 0.8450006684154654, 'colsample_bytree': 0.8875084020743679}


[I 2023-10-04 12:37:07,423] Trial 1 finished with value: 0.7327888774186851 and parameters: {'n_estimators': 195, 'max_depth': 18, 'learning_rate': 0.004841990774581337, 'min_child_weight': 20, 'gamma': 0.9482291259579523, 'subsample': 0.8226381464741157, 'colsample_bytree': 0.5331141424011663}. Best is trial 0 with value: 0.887473525678869.


Trial 1: 0.7327888774186851
Params: {'n_estimators': 195, 'max_depth': 18, 'learning_rate': 0.004841990774581337, 'min_child_weight': 20, 'gamma': 0.9482291259579523, 'subsample': 0.8226381464741157, 'colsample_bytree': 0.5331141424011663}


...

Trial 17: 0.9163536753918793
Params: {'n_estimators': 108, 'max_depth': 17, 'learning_rate': 0.09973604167703974, 'min_child_weight': 17, 'gamma': 0.13351057987674883, 'subsample': 0.2166106115105469, 'colsample_bytree': 0.21580531919284776}


[I 2023-10-04 12:38:33,670] Trial 18 finished with value: 0.9651305610489861 and parameters: {'n_estimators': 161, 'max_depth': 20, 'learning_rate': 0.012863656763929164, 'min_child_weight': 14, 'gamma': 0.21748014880223387, 'subsample': 0.17171000095056022, 'colsample_bytree': 0.4351229387814561}. Best is trial 18 with value: 0.9651305610489861.


Trial 18: 0.9651305610489861
Params: {'n_estimators': 161, 'max_depth': 20, 'learning_rate': 0.012863656763929164, 'min_child_weight': 14, 'gamma': 0.21748014880223387, 'subsample': 0.17171000095056022, 'colsample_bytree': 0.4351229387814561}


[I 2023-10-04 12:38:39,266] Trial 19 finished with value: 0.7942743219337373 and parameters: {'n_estimators': 166, 'max_depth': 20, 'learning_rate': 0.004018140697820689, 'min_child_weight': 10, 'gamma': 0.39845959537245684, 'subsample': 0.3384067757527567, 'colsample_bytree': 0.46739408731522386}. Best is trial 18 with value: 0.9651305610489861.


Trial 19: 0.7942743219337373
Params: {'n_estimators': 166, 'max_depth': 20, 'learning_rate': 0.004018140697820689, 'min_child_weight': 10, 'gamma': 0.39845959537245684, 'subsample': 0.3384067757527567, 'colsample_bytree': 0.46739408731522386}


In [26]:
# Evaluation
print("\n\nXGB Classifier Model")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f" % (xgb_prec[1], xgb_rec[1], xgb_f1[1]))
micro_f1 = f1_score(y_test, xgb_preds, average='micro')
print("Micro-Average F1 Score:", micro_f1)
classification = classification_report(y_test, xgb_preds)
print("Classification Report: \n", classification)



XGB Classifier Model
Precision: 0.973 
Recall: 0.651 
F1 Score: 0.780
Micro-Average F1 Score: 0.9769390532120604
Classification Report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     13087
           1       0.97      0.65      0.78       876

    accuracy                           0.98     13963
   macro avg       0.97      0.82      0.88     13963
weighted avg       0.98      0.98      0.97     13963



In [27]:
# Best params
xgb_best_params

{'n_estimators': 161,
 'max_depth': 20,
 'learning_rate': 0.012863656763929164,
 'min_child_weight': 14,
 'gamma': 0.21748014880223387,
 'subsample': 0.17171000095056022,
 'colsample_bytree': 0.4351229387814561}